### Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

- 절차
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

In [2]:
# 1. 동이름으로 위도 경도 구하기

In [3]:
addr = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.556785583496094, 126.9013442993164)

In [4]:
# 2. 위도 경도로 geohash 알아내기

In [5]:
import geohash2

In [6]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

In [7]:
# 3. geohash로 매물 아이디 가져오기

In [8]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
datas = response.json()["items"]
# len(datas), datas[0]
ids = [data["item_id"] for data in datas]
len(ids), ids[:5]

(872, [30543359, 30453086, 30462357, 30519078, 30524930])

In [9]:
# 4. 매물 아이디로 매물 정보 가져오기

In [10]:
# 1000개 넘어가면 나눠서 수집해야 함
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids
}
response = requests.post(url, params)
response

<Response [200]>

In [11]:
datas = response.json()["items"]
df = pd.DataFrame(datas)
df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
870,None,30518728,https://ic.zigbang.com/ic/items/30518728/1.jpg,월세,월세,1000,42,14.43,"{'m2': 14.43, 'p': '4.4'}","{'m2': 14.43, 'p': '4.4'}",...,False,True,원룸,[],서울시 마포구 중동,None,None,5,2022-02-07T19:08:42+09:00,False
871,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",...,False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False


In [12]:
# 필요한 컬럼만 필터링
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor", "address1", "manage_cost"]
filtered_column_df = df[columns]
filtered_column_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
870,30518728,월세,1000,42,14.43,3,4,서울시 마포구 중동,5
871,30455640,전세,28000,0,33.58,6,6,서울시 마포구 중동,3


In [13]:
# 주소에 망원동이 있는 데이터만 필터링
result_df = filtered_column_df[filtered_column_df["address1"].str.contains("망원동")].reset_index(drop=True)
result_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
98,30486794,월세,200,60,19.83,2,3,서울시 마포구 망원동,5
99,30448084,전세,25000,0,50.00,4,4,서울시 마포구 망원동,5
